# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [3]:
# your code here
import pandas as pd
from sklearn.preprocessing import StandardScaler

tic_tac_toe = pd.read_csv('tic-tac-toe.csv')

mapping = {'x': 1, 'o': -1, 'b': 0}
tic_tac_toe.replace(mapping, inplace=True)

X = tic_tac_toe.iloc[:, :-1]  # All columns except the last one
y = tic_tac_toe.iloc[:, -1]   # Last column (class)

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


/Users/slevin/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/slevin/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/slevin/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/slevin/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinsta

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [4]:
# your code here
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42)

model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

# Save the model
model.save("tic-tac-toe.model")


Epoch 1/50
17/17 [==============================] - 0s 7ms/step - loss: 0.6802 - accuracy: 0.5728 - val_loss: 0.6216 - val_accuracy: 0.6493
Epoch 2/50
17/17 [==============================] - 0s 2ms/step - loss: 0.6194 - accuracy: 0.6791 - val_loss: 0.5805 - val_accuracy: 0.7313
Epoch 3/50
17/17 [==============================] - 0s 2ms/step - loss: 0.5761 - accuracy: 0.7276 - val_loss: 0.5517 - val_accuracy: 0.7463
Epoch 4/50
17/17 [==============================] - 0s 2ms/step - loss: 0.5348 - accuracy: 0.7537 - val_loss: 0.5232 - val_accuracy: 0.7612
Epoch 5/50
17/17 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7799 - val_loss: 0.5014 - val_accuracy: 0.7761
Epoch 6/50
17/17 [==============================] - 0s 2ms/step - loss: 0.4551 - accuracy: 0.7948 - val_loss: 0.4775 - val_accuracy: 0.7836
Epoch 7/50
17/17 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8153 - val_loss: 0.4583 - val_accuracy: 0.7836
Epoch 8/50
17/17 [==

INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [5]:
# your code here
from tensorflow.keras.models import load_model
import numpy as np

loaded_model = load_model("tic-tac-toe.model")

num_samples_to_predict = 5  # Change this number as needed
random_indices = np.random.choice(X_test.shape[0], num_samples_to_predict, replace=False)
random_samples = X_test[random_indices]

predictions = loaded_model.predict(random_samples)
rounded_predictions = np.round(predictions)

for i, idx in enumerate(random_indices):
    print(f"Sample {i+1}:")
    print(f"Predicted: {rounded_predictions[i]}, Actual: {y_test.iloc[idx]}")
    print()

1/1 [==============================] - 0s 40ms/step
Sample 1:
Predicted: [0.], Actual: False

Sample 2:
Predicted: [1.], Actual: True

Sample 3:
Predicted: [0.], Actual: False

Sample 4:
Predicted: [1.], Actual: True

Sample 5:
Predicted: [1.], Actual: True



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [6]:
# your code here
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
loaded_model = load_model("tic-tac-toe.model")

# Make predictions on a few random rows from the test dataset
num_samples_to_predict = 5  # Change this number as needed
random_indices = np.random.choice(X_test.shape[0], num_samples_to_predict, replace=False)
random_samples = X_test[random_indices]

# Predict
predictions = loaded_model.predict(random_samples)
rounded_predictions = np.round(predictions)

# Compare predictions with actual labels
for i, idx in enumerate(random_indices):
    print(f"Sample {i+1}:")
    print(f"Predicted: {rounded_predictions[i]}, Actual: {y_test.iloc[idx]}")
    print()


1/1 [==============================] - 0s 30ms/step
Sample 1:
Predicted: [1.], Actual: True

Sample 2:
Predicted: [1.], Actual: True

Sample 3:
Predicted: [1.], Actual: True

Sample 4:
Predicted: [1.], Actual: True

Sample 5:
Predicted: [1.], Actual: True



**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
"""
Improving model performance involves a lot of trial an error. By adjusting the architecture by adding more layers seemed to have a positive impact in this case. 
This model was able to create potentially more complex patterns in the data, which can be beneficial for solving intricate problems.
"""
